<a href="https://colab.research.google.com/github/VivekSM1992/HumanActivityRecognition/blob/main/har.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
import random
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from numpy import dstack
from tensorflow.keras.utils import to_categorical
import math
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from numpy import mean
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


import os

list_dir=os.listdir("/content/drive/MyDrive/A_DeviceMotion_data")
print(list_dir)

['ups_12', 'std_14', 'wlk_15', 'sit_13', 'wlk_8', 'wlk_7', 'sit_5', 'ups_3', 'ups_4', 'std_6', 'dws_1', 'jog_9', 'dws_11', 'jog_16', 'dws_2']


In [19]:
Folders = glob('/content/drive/MyDrive/A_DeviceMotion_data/*_*')

In [20]:
csv_files_train=[]
csv_files_test=[]
test_files_index=[]
all_csv_files_train=[]
all_csv_files_test=[]
for item in Folders:
    files_list=os.listdir(item)
    total_file_size=len(files_list)
    train_file_size=math.floor(total_file_size*0.75)
    random.seed(0)
    train_files_index=random.sample(range(total_file_size), train_file_size)
    
    for j in range(0,total_file_size):
        if j not in train_files_index:
            test_files_index.append(j)
    #print(test_files_index)
    #print(train_files_index)
    
    
    for index in test_files_index:
        file=files_list[index]
        test_file=glob(item+'/'+str(file))
        csv_files_test.append(test_file)
    
    for index in train_files_index:
        file=files_list[index]
        train_file=glob(item+'/'+str(file))
        csv_files_train.append(train_file)
     
    test_files_index.clear()
    train_files_index.clear()
        
for i in csv_files_train:
        all_csv_files_train=all_csv_files_train+i


for i in csv_files_test:
        all_csv_files_test=all_csv_files_test+i

In [21]:
def prepare_data(X,groundtruth):
    WL=128
    overlap=64
    indexi=0
    indexf=indexi+WL-1
    container_X=[]
    #container_Y=[]
    num_timesteps,num_features = X.shape
    #print(num_timesteps)
    num_windows=(num_timesteps-WL)// (WL-overlap)
    X_wind=np.zeros([num_windows,WL], dtype = float)
    Y_wind=np.full([num_windows, 1],groundtruth,dtype = int)
    #print(num_features)
    
    
    for features in range(0,num_features):
        indexi=0
        indexf=indexi+WL-1
        #print(X)
        for windows in range(0,num_windows):
            X_wind[windows,:]=X.iloc[indexi:(indexf+1),features]
            
            indexi=indexf-overlap+1
            #print(indexi)
            indexf=indexi+WL-1
        container_X.append(X_wind)
        #container_Y.append(Y_wind)
    container_X=dstack(container_X)
    #container_Y=dstack(container_Y)
    return container_X,Y_wind,num_windows

In [22]:
def process_data(csv_files):
    X_list=[]
    Y_list=[]
    Total_num_windows=0
    daddy_frame = pd.DataFrame()
    #print(X_list)
    for csvs in csv_files:
        for items in gt_list:
            if (str(csvs).find(items) != -1):
                break
        for subjects in sub_list:
            if (str(csvs).find(subjects) != -1):
                break
        temp_df=pd.read_csv(csvs)
        #print(temp_df)
        #temp_df=temp_df.loc[0:1028, :]
        temp_df=temp_df.drop(labels='Unnamed: 0', axis=1)
        temp_df['ang_res']=temp_df['rotationRate.x'] ** 2+ temp_df['rotationRate.y'] ** 2+ temp_df['rotationRate.z'] ** 2
        temp_df['ang_res']=temp_df['ang_res'].apply(np.sqrt)
        temp_df['linear_res']=temp_df['userAcceleration.x'] ** 2+ temp_df['userAcceleration.y'] ** 2+ temp_df['userAcceleration.z'] ** 2
        temp_df['linear_res']=temp_df['linear_res'].apply(np.sqrt)
    
        temp_df_scale=temp_df.loc[:,['rotationRate.x','rotationRate.y','rotationRate.z','linear_res']]
        scaler = MinMaxScaler((-1,1))
        scaler.fit(temp_df_scale)
        scaled_X=scaler.transform(temp_df_scale)
        
        for key, value in gt_list_dict.items():
            if key == items:
                items=value
        #print(temp_df_scale)
        process_data_X,process_data_Y,num_windows=prepare_data(temp_df_scale,items)
        X_list.append(process_data_X)
        Y_list.append(process_data_Y)
        Total_num_windows=Total_num_windows+num_windows
        temp_df['groundtruth']=items
        temp_df['subjects']=subjects
        #print(temp_df.head())
        #print((temp_df.count))
        daddy_frame=daddy_frame.append(temp_df)
    X_list=np.array(X_list)
    Y_list=np.array(Y_list)
    return daddy_frame,X_list,Y_list,Total_num_windows

In [23]:
sub_list=[]
for i in range(1,25):
    string='sub_'+str(i)+'.csv'
    sub_list.append(string)
gt_list=['dws','jog','sit','ups','wlk','std']
gt_list_dict={'dws':0,'jog':1,'sit':2,'ups':3,'wlk':4,'std':5}

def load_dataset():
    
    master_frame_train = pd.DataFrame()
    master_frame_test = pd.DataFrame()
    

    master_frame_train,Train_X,Train_Y,Total_num_windows_train=process_data(all_csv_files_train)
    
    master_frame_test,Test_X,Test_Y,Total_num_windows_test=process_data(all_csv_files_test)
    #print(master_frame_test)


   # print(Train_Y[1].shape)
    WL=128
    num_features=4
    num_windows_train=0
    num_windows_test=0
    Train_X_array=np.zeros([Total_num_windows_train,WL,num_features], dtype = float)
    Train_Y_array=np.zeros([Total_num_windows_train,1], dtype = int)
    Test_X_array=np.zeros([Total_num_windows_test,WL,num_features], dtype = float)
    Test_Y_array=np.zeros([Total_num_windows_test,1], dtype = int)
    for i in range(0,Train_X.size):
        
        for j in range(0,Train_X[i].shape[0]):
            
            Train_X_array[num_windows_train+j]=Train_X[i][j]
            Train_Y_array[num_windows_train+j]=Train_Y[i][j]
        num_windows_train=num_windows_train+(j+1)
    
    for i in range(0,Test_X.size):
        #print(i)
        for j in range(0,Test_X[i].shape[0]):
            #print(j)
            Test_X_array[num_windows_test+j]=Test_X[i][j]
            Test_Y_array[num_windows_test+j]=Test_Y[i][j]
        num_windows_test=num_windows_test+(j+1)
    print(Train_Y_array.shape,Test_Y_array.shape)    
    # one hot encode y
    Train_Y_gt=Train_Y_array
    Train_Y_array = to_categorical(Train_Y_array)
    Test_Y_gt=Test_Y_array
    Test_Y_array = to_categorical(Test_Y_array)
    print(Train_X_array.shape, Train_Y_array.shape, Test_X_array.shape, Test_Y_array.shape)
    return Train_X_array, Train_Y_array, Test_X_array, Test_Y_array,Test_Y_gt,Train_Y_gt

In [ ]:
 from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy,test_Y_gt,train_Y_gt,type_model):
    verbose, epochs, batch_size = 1, 50, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    type_model="CNN"
    if(type_model=="CNN"):
        model.add(Conv1D(filters=64, kernel_size=16, activation='relu', input_shape=(n_timesteps,n_features)))
        model.add(Conv1D(filters=32, kernel_size=16, activation='relu'))
  #model.add(Dropout(0.5))
  #model.add(Dropout(0.5))
        model.add(MaxPooling1D(pool_size=4))
  #model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
  #model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
  #model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
    else:
        model.add(LSTM(64, input_shape=(n_timesteps,n_features)))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=2)
 
    yhat_probs = model.predict(testX, verbose=0)
    #print(yhat_probs)
  # predict crisp classes for test set
    yhat_classes = np.argmax(model.predict(testX), axis=-1)
    print(yhat_classes)
    target_names = ['Walking Downstairs','Jogging','Sitting', 'Walking Upstairs','Walking','Standing']
    print(classification_report(test_Y_gt, yhat_classes, target_names=target_names))
    confusion = confusion_matrix(test_Y_gt, yhat_classes)
    print('Confusion Matrix\n')
    print(confusion)
    return accuracy
 
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run an experiment
def run_experiment(repeats=2):
	# load data
	trainX, trainy, testX, testy,test_Y_gt,train_Y_gt = load_dataset()
	print(trainX)
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy,test_Y_gt,train_Y_gt,"LSTM")
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

 
# run the experiment
run_experiment()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

(15783, 1) (5396, 1)
(15783, 128, 4) (15783, 6) (5396, 128, 4) (5396, 6)
[[[2.64753792 2.64753792 2.64753792 2.64753792]
  [0.87340286 0.87340286 0.87340286 0.87340286]
  [1.17312502 1.17312502 1.17312502 1.17312502]
  ...
  [0.6367619  0.6367619  0.6367619  0.6367619 ]
  [0.53612124 0.53612124 0.53612124 0.53612124]
  [0.43328612 0.43328612 0.43328612 0.43328612]]

 [[0.89999376 0.89999376 0.89999376 0.89999376]
  [0.80182436 0.80182436 0.80182436 0.80182436]
  [0.9123571  0.9123571  0.9123571  0.9123571 ]
  ...
  [1.01925185 1.01925185 1.01925185 1.01925185]
  [0.89933323 0.89933323 0.89933323 0.89933323]
  [0.72555377 0.72555377 0.72555377 0.72555377]]

 [[0.56623496 0.56623496 0.56623496 0.56623496]
  [0.54740049 0.54740049 0.54740049 0.54740049]
  [0.6496284  0.6496284  0.6496284  0.6496284 ]
  ...
  [2.55298546 2.55298546 2.55298546 2.55298546]
  [0.43676464 0.43676464 0.43676464 0.43676464]
  [0.16718532 0.16718532 0.16718532 0.16718532]]

 ...

 [[0.59164619 0.59164619 0.591646